# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.234523616300                   -0.50    8.0
  2   -7.250206895731       -1.80       -1.39    1.0
  3   -7.251310322569       -2.96       -2.09    4.0
  4   -7.251296233480   +   -4.85       -2.27    4.0
  5   -7.251331119314       -4.46       -2.71    3.0
  6   -7.251336448096       -5.27       -2.97    2.0
  7   -7.251338762758       -5.64       -3.92    1.0
  8   -7.251338788426       -7.59       -4.06    4.0
  9   -7.251338798308       -8.01       -4.71    3.0
 10   -7.251338798574       -9.58       -5.02    1.0
 11   -7.251338798701       -9.90       -5.78    3.0
 12   -7.251338798704      -11.47       -6.15    3.0
 13   -7.251338798704      -12.57       -6.60    2.0
 14   -7.251338798705      -13.13       -7.03    3.0
 15   -7.251338798705      -13.94       -7.81    3.0
 16   -7.251338798705      -14.57       -8.36    3.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.06646460493158168
[ Info: Arnoldi iteration step 2: normres = 0.22600517225561173
[ Info: Arnoldi iteration step 3: normres = 0.6392267341937111
[ Info: Arnoldi iteration step 4: normres = 0.6830771901171617
[ Info: Arnoldi iteration step 5: normres = 0.6401841040976396
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.51e-01, 7.04e-02, 5.37e-01, 2.28e-01, 4.30e-02)
[ Info: Arnoldi iteration step 6: normres = 0.20757385266806738
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.62e-02, 1.41e-01, 8.87e-02, 1.00e-01, 1.55e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09471113535146315
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.06e-03, 1.14e-02, 5.89e-03, 4.50e-02, 7.91e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10511824228304247
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (9.18e